In [16]:
import pandas as pd 
import numpy as np

from googletrans import Translator
translator = Translator()

def to_float(x):
    if type(x) == str and "," in x:
        x = x.replace(",", ".")
        return float(x)
    elif type(x) == str and x.isnumeric():
        return eval(x)
    return x

In [17]:
def clean_data(clean_guide):
    filename = clean_guide["filename"]
    genesis_table = clean_guide["Table"],
    column_names = clean_guide["Columns"]
    column_names = column_names.split(", ")
    skiprows = clean_guide['Read Lines'][0] - 1
    nrows = clean_guide['Read Lines'][1] - skiprows
    na_values = clean_guide['NA']
    to_translate = clean_guide["To Translate"]

    df = pd.read_csv(filename, sep=";", encoding="latin-1", header=None, names=column_names, skiprows=skiprows, nrows=nrows, na_values=na_values)

    # Convert to numeric
    for col in column_names[clean_guide['to_numeric']:]:
        df[col] = df[col].apply(lambda x: to_float(x))

    if to_translate:
        for k, v in to_translate.items():
            if v: # can be optimized
                unique_vals = list(df[k].unique())
                translated_unique_vals = [translator.translate(x).text for x in unique_vals]
                mappings = dict(zip(unique_vals, translated_unique_vals))
                df[k] = df[k].apply(lambda x: mappings[x])
                df[k] = df[k].str.title()
            else:
                df[k] = df[k].apply(lambda x: translator.translate(x).text)
    
    return df

In [24]:
d1 = {
    "filename": "Sales Packaging-companies-end consumers-federal states-years.csv",
    "Table": "32136-0010",
    "Columns": "Year, State, Company, Sales packaging. v. private end users (1000t), Sales packaging. v. private end use per inhabitant (kg)",
    "Read Lines": [7, 182],
    "To Translate": {'State':1},
    "NA": [".", "-", "x"],
    "to_numeric": 2
}
d2 = {
    "filename": "Sales Packaging-companies-end consumers-years-return types.csv",
    "Table": "32136-0002",
    "Columns": "Year, Return Method, Unit, Sales Packaging After Sorting (Industrial Solutions), Sales Packaging After Sorting (System Operators), Overall Sales Packaging After Sorting",
    "Read Lines": [9, 63],
    "NA": [".", "x", "-"],
    "To Translate": {'Return Method': 0, 'Unit': 1},
    "to_numeric": 3
}
d3 = {
    "filename": "Sales Packaging-companies-end consumers-years.csv",
    "Table": "32136-0001",
    "Columns": "KPI, Unit, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020",
    "Read Lines": [8,12],
    "NA": [".", "-", "x"],
    "To Translate": {"KPI": 0, "Unit": 1},
    "to_numeric": 2,
    "pivot": True
}
d4 = {
    "filename": "Sales Packaging-end consumers-federal states-years-type of sales.csv",
    "Table": "32136-0011",
    "Columns": "Year, State, Mixed packaging, Packaging made of paper & cardboard & carton, Mixed glass (colored & mixed glass), Glass collected separately in color, Plastics collected separately, Metals collected separately, Composites collected separately, Total",
    "Read Lines": [8, 183],
    "NA": [".", "-"],
    "To Translate": {"State":1},
    "to_numeric": 2,
    "pivot":True
}
d5 = {
    "filename": "Sales Packaging-end consumers-years-return types-type of sales packaging.csv",
    "Table": "32136-0003",
    "Columns": "Year, Sales packaging. v. private end consumers, Industry solutions (1000 t), System operators (1000 t), Overall (1000 t)",
    "Read Lines": [10, 97],
    "NA": [".", "-"],
    "To Translate": {"Sales packaging. v. private end consumers": 0},
    "to_numeric": 2,
    "pivot":True
}
d6 = {
    "filename": "Sales Packaging-sorted-years-return types-recycling types.csv",
    "Table": "32136-0006",
    "Columns": "Year, Recycling Type, Sales packaging after sorting - Industry solutions (1000t), Sales packaging after sorting - System operators (1000t), Overall sales packaging after sorting (1000t)",
    "Read Lines": [9, 74],
    "NA": [".", "-"],
    "To Translate": {"Recycling Type":0},
    "to_numeric": 2,
    "pivot": True
}
d7 = {
    "filename": "Sales Packaging-years-types of material-types of recycling.csv",
    "Table": "32136-0005",
    "Columns": "Year, Recycling Material, Sales packaging after sorting (1000 t) - Material recycling, Sales packaging after sorting (1000 t) - Other forms of material recycling, Sales packaging after sorting (1000 t) - Energy recycling, Sales packaging after sorting (1000 t) - Other forms of recycling, Sales packaging after sorting (1000 t) - Other disposal, Total Sales packaging after sorting (1000 t)",
    "Read Lines": [9, 107],
    "NA": [".", "-"],
    "To Translate": {"Recycling Type":0},
    "to_numeric": 2,
    "pivot": True
}
d8 = {
    "filename": "Sales Packaging-years-types of material.csv",
    "Table": "32136-0004",
    "Columns": "Year, Material Type, Sales packaging after sorting (1000t), Handing over of sales packaging. n.d. Sort of abroad (1000t)",
    "Read Lines": [8, 106],
    "NA": [".", "-"],
    "To Translate": {'Material Type': 0},
    "to_numeric": 2
}

In [4]:
waste_df1 = clean_data(d1)
waste_df1.head()

,Year,State,Company,Sales packaging. v. private end users (1000t),Sales packaging. v. private end use per inhabitant (kg)
0,2010,Baden-Württemberg,NaN,746.9,69
1,2010,Bayern,NaN,800.1,64
2,2010,Berlin,NaN,208.1,60
3,2010,Brandenburg,NaN,205.6,82
4,2010,Bremen,NaN,55.1,83


In [8]:
waste_df1.drop(columns=['Company'], inplace=True)

In [9]:
waste_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 4 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   Year                                                     176 non-null    int64  
 1   State                                                    176 non-null    object 
 2   Sales packaging. v. private end users (1000t)            176 non-null    float64
 3   Sales packaging. v. private end use per inhabitant (kg)  176 non-null    int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 5.6+ KB


In [10]:
waste_df1.to_csv(f"cleaned_{d1['Table']}_Sales-Packaging-{d1['filename']}", index=False)

In [ ]:
waste_df2 = clean_data(d2)
waste_df2.head()

In [21]:
waste_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 6 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Year                                                  55 non-null     int64  
 1   Return Method                                         55 non-null     object 
 2   Unit                                                  55 non-null     object 
 3   Sales Packaging After Sorting (Industrial Solutions)  44 non-null     float64
 4   Sales Packaging After Sorting (System Operators)      44 non-null     float64
 5   Overall Sales Packaging After Sorting                 55 non-null     float64
dtypes: float64(3), int64(1), object(2)
memory usage: 2.7+ KB


In [22]:
waste_df2.to_csv(f"cleaned_{d2['Table']}_Sales-Packaging-{d2['filename']}", index=False)

In [25]:
waste_df3 = clean_data(d3)
waste_df3.head()

,KPI,Unit,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Pursue,Number,40.0,41.0,41.0,41.0,42.0,32.0,32.0,28.0,21.0,20.0,16.0
1,Captured sales packaging. v. private end users,1000 T,5772.6,5913.0,6051.9,6382.2,6374.6,5598.5,5647.3,5694.5,5668.2,5949.8,6503.0
2,Req. Sales packaging. v. private end use per i...,Kg,71.0,74.0,75.0,79.0,79.0,68.0,68.0,69.0,68.0,72.0,78.0
3,Sales packaging after sorting,1000 T,5693.9,5711.0,5856.8,6196.6,6063.7,5518.4,5450.9,5391.8,5403.9,5635.2,6436.0
4,Submission of sales packages n.d. Sort of abroad,1000 T,326.1,363.5,378.4,325.8,320.7,313.0,335.7,352.7,360.9,472.0,568.5


In [27]:
waste_df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   KPI     5 non-null      object 
 1   Unit    5 non-null      object 
 2   2010    5 non-null      float64
 3   2011    5 non-null      float64
 4   2012    5 non-null      float64
 5   2013    5 non-null      float64
 6   2014    5 non-null      float64
 7   2015    5 non-null      float64
 8   2016    5 non-null      float64
 9   2017    5 non-null      float64
 10  2018    5 non-null      float64
 11  2019    5 non-null      float64
 12  2020    5 non-null      float64
dtypes: float64(11), object(2)
memory usage: 648.0+ bytes


In [28]:
waste_df3.to_csv(f"cleaned_{d3['Table']}_Sales-Packaging-{d3['filename']}", index=False)

In [26]:
pd.wide_to_long(waste_df3, stubnames=[''], i=['KPI', 'Unit'], j='Year')

KPI                                                Unit   Year        
Pursue                                             Number 2010    40.0
                                                          2011    41.0
                                                          2012    41.0
                                                          2013    41.0
                                                          2014    42.0
                                                          2015    32.0
                                                          2016    32.0
                                                          2017    28.0
                                                          2018    21.0
                                                          2019    20.0
                                                          2020    16.0
Captured sales packaging. v. private end users     1000 T 2010  5772.6
                                                          2011  5913.0
                                                          2012  6051.9
                                                          2013  6382.2
                                                          2014  6374.6
                                                          2015  5598.5
                                                          2016  5647.3
                                                          2017  5694.5
                                                          2018  5668.2
                                                          2019  5949.8
                                                          2020  6503.0
Req. Sales packaging. v. private end use per in... Kg     2010    71.0
                                                          2011    74.0
                                                          2012    75.0
                                                          2013    79.0
                                                          2014    79.0
                                                          2015    68.0
                                                          2016    68.0
                                                          2017    69.0
                                                          2018    68.0
                                                          2019    72.0
                                                          2020    78.0
Sales packaging after sorting                      1000 T 2010  5693.9
                                                          2011  5711.0
                                                          2012  5856.8
                                                          2013  6196.6
                                                          2014  6063.7
                                                          2015  5518.4
                                                          2016  5450.9
                                                          2017  5391.8
                                                          2018  5403.9
                                                          2019  5635.2
                                                          2020  6436.0
Submission of sales packages n.d. Sort of abroad   1000 T 2010   326.1
                                                          2011   363.5
                                                          2012   378.4
                                                          2013   325.8
                                                          2014   320.7
                                                          2015   313.0
                                                          2016   335.7
                                                          2017   352.7
                                                          2018   360.9
                                                          2019   472.0
                                                          2020   568.5